In [37]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [33]:
import json
from typing import List
from uuid import UUID
from langchain.tools import BaseTool
from pydantic import BaseModel, TypeAdapter
class CustomDataReferenceTool(BaseTool):

    class Data(BaseModel):
        id: str
        name: str
        age: int
        favorites: List[str]

    name = "Custom DB reference"
    description = "Useful when you need to get the data from custom database."
    data_list: List[Data] = []

    def __init__(self, **data):
        if "data_list" not in data:
            with open("./sample_db.json", "r") as f:
                json_string = json.loads(f.read())
                data["data_list"] = TypeAdapter(List[self.Data]).validate_python(json_string)
        super(CustomDataReferenceTool, self).__init__(**data)


    def _run(self, name: str) -> List[str]:
        for data in self.data_list:
            if name == data.name:
                return data.favorites
        raise ValueError(f"No data with name={name} found.")

    def _arun(self, name: str) -> str:
        raise NotImplementedError()

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name="gpt-3.5-turbo"
)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,
    return_messages=True
)

tools = [CustomDataReferenceTool()]

In [40]:
from tabnanny import verbose
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=conversational_memory
)

In [41]:
agent("What's Mio's favorite food?")



> Entering new AgentExecutor chain...
{
    "action": "Custom DB reference",
    "action_input": "Mio"
}
Observation: ['Okonomiyaki']
Thought:{
    "action": "Final Answer",
    "action_input": "Mio's favorite food is Okonomiyaki."
}

> Finished chain.


{'input': "What's Mio's favorite food?",
 'chat_history': [],
 'output': "Mio's favorite food is Okonomiyaki."}

In [42]:
agent("What Yui's favorite food?")



> Entering new AgentExecutor chain...
{
    "action": "Custom DB reference",
    "action_input": "Yui"
}
Observation: ['Daifuku', 'Katsu', 'Sushi']
Thought:{
    "action": "Final Answer",
    "action_input": "Yui's favorite food is Daifuku, Katsu, and Sushi."
}

> Finished chain.


{'input': "What Yui's favorite food?",
 'chat_history': [HumanMessage(content="What's Mio's favorite food?"),
  AIMessage(content="Mio's favorite food is Okonomiyaki.")],
 'output': "Yui's favorite food is Daifuku, Katsu, and Sushi."}